In [18]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.cross_validation import train_test_split

from load import all_subreddits_data, tv_subreddits_data


In [2]:
# data is of the form {class_label: list of documents}
all_data = all_subreddits_data()
tv_data = tv_subreddits_data()

10


In [27]:
def tt_split(data, test_size=.1):
    """Splits a dictionary {class_label: list of documents}"""
    """into two dictionaries of the same shape"""
    train_data = {}; test_data = {}
    for label, docs in data.iteritems():
        train, test = train_test_split(docs, test_size=test_size)
        train_data[label] = train
        test_data[label] = test
    return train_data, test_data
        
        

In [39]:
def tokens_to_vocab(class_tokens):
    """{class_label : list of tokenized documents} -> vocab"""
    vocab = set([])
    for _class, tokenized_docs in class_tokens.iteritems():
        for d in tokenized_docs:
            vocab = vocab.union(set(d))
    return {word: i for i, word in enumerate(vocab)}
        

def word_tokenize_doc(doc):
    """Word tokenize a single document"""
    to_remove = set(['http', 'faq', 'https', 'amp','source', 'deletion', 'sfw',
              'nsfw', 'gt', 'gon', 'na', 'delete', 'comment', 'profile'])
    def _filter(w):
        return all([w.isalnum(), w not in stopwords.words('english'), w not in to_remove])
    tokens = word_tokenize(doc)
    tokens = filter(_filter, tokens)
    return tokens

def tokenize_all_words(data):
    """basic get_tokens method"""
    """{class_label: list of documents} ->""" 
    """{class_label : list of tokenized documents}"""
    for c, docs in data.iteritems():
        data[c] = map(word_tokenize_doc, docs)
    return data

test_data = {
    'c1': ['aa bb c', 'b d'],
    'c2': ['d e f', 'e g']
}
test_tokens = tokenize_all_words(test_data)
tokens_to_vocab(test_tokens)


{'b': 1, 'c': 0, 'd': 3, 'e': 2, 'f': 5, 'g': 4}

In [32]:
def get_lda_topics(train_tokens, vocab):
    """{class_label: list of tokenized docs} ->"""
    """{class_label: list of topic distributions from LDA"""
    pass

In [29]:
def run_model(data, get_tokens=tokenize_all_words,
              get_topics = get_lda_topics):
    """Vectorizes, topic models, classifies, and returns score"""
    train, test = tt_split(data)
    train_tokens = get_tokens(train)
    vocab = tokens_to_vocab(train_tokens)
    class_topic_dists = get_topics
    return vocab
    
    # counts = vectorize(train, vocab)

vocab = run_model(all_data)

In [34]:
print vocab

set(['A', 'a', 'c', 'e', 'd', '0', 'i', 'm', 'l', 'n', 'p', 's', '2', '7', 'v'])
